# 생태통로-경지계 간 최단 거리
경지계 데이터가 커 두 개로 나눴으므로, 각 데이터에 대해 계산한 거리를 비교해 더 작은 것을 저장

In [2]:
!pip install pymysql

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 995 kB/s 


In [3]:
from sqlalchemy import create_engine
import pymysql
import numpy as np
import pandas as pd

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 데이터 로드

In [5]:
endpoint = 'dao.cdqd2bwz9q4s.ap-northeast-2.rds.amazonaws.com'
schema_name = 'dao'
db_connection_str = 'mysql+pymysql://admin:ekfkawnl@{}/{}'.format(endpoint, schema_name )
try :
    db_connection = create_engine(db_connection_str)
    conn = db_connection.connect()
except :
    print('fail to connect db')

In [6]:
def sql_to_df(table_name):
    sql = "SELECT * FROM {}".format(table_name)
    df = pd.read_sql(sql, db_connection)
    return df

In [7]:
mdyr = sql_to_df("MainData_inner_예림")
mdyr

,번호,주변 로드킬 빈도,산책로까지의 최단 거리(km),최단 거리 표지판의 주행제한속도(km),농가까지의 거리(km)
0,1495,0,32.993820,NaN,0.223706
1,1277,0,189.603501,NaN,0.303690
2,1278,0,172.236247,NaN,0.011154
3,1279,0,172.092457,NaN,0.012767
4,1090,5,140.219401,NaN,0.183505
...,...,...,...,...,...
492,1089,5,140.166888,NaN,0.287976
493,1087,5,140.681388,NaN,0.119958
494,1088,5,140.066912,NaN,0.116240
495,1091,0,145.890079,NaN,0.023978


In [8]:
path = '/content/drive/MyDrive/2022 데이터 청년 캠퍼스/프로젝트/데이터/'
farm1 = pd.read_csv(path + '경지계1-3.csv')
farm2 = pd.read_csv(path + '경지계4-7.csv')

In [9]:
farm1.head(3)

,번호,위도_십진,경도_십진,생태통로타,차선_수,등산객이용,생태통로효,이용확인종,이용확인,면적_m^2,너비_m,길이_m,join_UFID,join_DIVI,join_SCLS,join_FMTA,join_layer,join_path,distance
0,1495,35.36019,128.06375,0,1,1,3,NaN,0,NaN,NaN,NaN,VGT0100000037M3BF,FLD002,NaN,NaN,NGII_CDM_寃쎌�怨�3 �� Z_NGII_N3A_D0010000_C.shp,/vsizip/C:/Users/tkfkd/Downloads/NGII_CDM_寃쎌�怨...,20874.160929
1,1277,37.03408,128.40661,1,0,0,3,NaN,0,NaN,NaN,NaN,VGT010000000CPZX2,FLD002,NaN,NaN,NGII_CDM_寃쎌�怨�1 �� Z_NGII_N3A_D0010000_A.shp,/vsizip/C:/Users/tkfkd/Downloads/NGII_CDM_寃쎌�怨...,303.690012
2,1278,36.89832,128.30563,1,0,0,1,고라니 멧돼지 파충류,3,NaN,NaN,NaN,VGT010000000R9IM9,FLD002,NaN,NaN,NGII_CDM_寃쎌�怨�1 �� Z_NGII_N3A_D0010000_A.shp,/vsizip/C:/Users/tkfkd/Downloads/NGII_CDM_寃쎌�怨...,11.153787


In [10]:
# 필요 없는 컬럼 drop
farm1.drop(['join_UFID',	'join_DIVI',	'join_SCLS',	'join_FMTA',	'join_layer',	'join_path'], axis=1, inplace=True)
farm2.drop(['join_UFID',	'join_DIVI',	'join_SCLS',	'join_FMTA',	'join_layer',	'join_path'], axis=1, inplace=True)

In [11]:
farm1

,번호,위도_십진,경도_십진,생태통로타,차선_수,등산객이용,생태통로효,이용확인종,이용확인,면적_m^2,너비_m,길이_m,distance
0,1495,35.36019,128.06375,0,1,1,3,NaN,0,NaN,NaN,NaN,20874.160929
1,1277,37.03408,128.40661,1,0,0,3,NaN,0,NaN,NaN,NaN,303.690012
2,1278,36.89832,128.30563,1,0,0,1,고라니 멧돼지 파충류,3,NaN,NaN,NaN,11.153787
3,1279,36.89687,128.30581,1,0,0,1,고라니 멧돼지 파충류,3,NaN,NaN,NaN,12.767402
4,1090,36.71336,127.42200,0,2,5,1,개 고양이 다람쥐,3,312.74,8.23,38.0,183.504631
...,...,...,...,...,...,...,...,...,...,...,...,...,...
492,1089,36.71278,127.42056,0,2,0,0,고라니 멧돼지,2,674.31,13.30,50.7,287.976255
493,1087,36.71722,127.41778,0,1,5,0,고라니 멧돼지,2,526.32,13.60,38.7,119.958413
494,1088,36.71222,127.42528,0,1,5,0,고라니 멧돼지,2,674.31,13.30,50.7,116.239892
495,1091,36.76750,127.47333,0,0,0,0,멧돼지 고라니,2,140.00,7.00,20.0,23.977596


In [12]:
farm2

,번호,위도_십진,경도_십진,생태통로타,차선_수,등산객이용,생태통로효,이용확인종,이용확인,면적_m^2,너비_m,길이_m,distance
0,1495,35.36019,128.06375,0,1,1,3,NaN,0,NaN,NaN,NaN,223.706303
1,1277,37.03408,128.40661,1,0,0,3,NaN,0,NaN,NaN,NaN,16115.052441
2,1278,36.89832,128.30563,1,0,0,1,고라니 멧돼지 파충류,3,NaN,NaN,NaN,940.797329
3,1279,36.89687,128.30581,1,0,0,1,고라니 멧돼지 파충류,3,NaN,NaN,NaN,779.243028
4,1090,36.71336,127.42200,0,2,5,1,개 고양이 다람쥐,3,312.74,8.23,38.0,34202.186858
...,...,...,...,...,...,...,...,...,...,...,...,...,...
492,1089,36.71278,127.42056,0,2,0,0,고라니 멧돼지,2,674.31,13.30,50.7,34263.209808
493,1087,36.71722,127.41778,0,1,5,0,고라니 멧돼지,2,526.32,13.60,38.7,33765.510016
494,1088,36.71222,127.42528,0,1,5,0,고라니 멧돼지,2,674.31,13.30,50.7,34337.983095
495,1091,36.76750,127.47333,0,0,0,0,멧돼지 고라니,2,140.00,7.00,20.0,28655.062778


In [13]:
(farm1['번호'] == farm2['번호']).sum()

497

## 농가까지의 최단 거리 저장

In [14]:
for i in range(len(farm1)):
  dist_m = min(farm1['distance'][i], farm2['distance'][i]) # 미터 단위 거리
  dist_km = dist_m / 1000 # 킬로미터 단위로 변환
  mdyr.loc[i, '농가까지의 거리(m)'] = dist_m
  mdyr.loc[i, '농가까지의 거리(km)'] = dist_km

In [15]:
mdyr

,번호,주변 로드킬 빈도,산책로까지의 최단 거리(km),최단 거리 표지판의 주행제한속도(km),농가까지의 거리(km),농가까지의 거리(m)
0,1495,0,32.993820,NaN,0.223706,223.706303
1,1277,0,189.603501,NaN,0.303690,303.690012
2,1278,0,172.236247,NaN,0.011154,11.153787
3,1279,0,172.092457,NaN,0.012767,12.767402
4,1090,5,140.219401,NaN,0.183505,183.504631
...,...,...,...,...,...,...
492,1089,5,140.166888,NaN,0.287976,287.976255
493,1087,5,140.681388,NaN,0.119958,119.958413
494,1088,5,140.066912,NaN,0.116240,116.239892
495,1091,0,145.890079,NaN,0.023978,23.977596


In [17]:
mdyr['농가까지의 거리(m)'].describe()

count      497.000000
mean       256.611815
std       1176.766676
min          0.000000
25%         27.725617
50%         67.285850
75%        138.246127
max      18861.855292
Name: 농가까지의 거리(m), dtype: float64

In [16]:
mdyr['농가까지의 거리(km)'].describe()

count    497.000000
mean       0.256612
std        1.176767
min        0.000000
25%        0.027726
50%        0.067286
75%        0.138246
max       18.861855
Name: 농가까지의 거리(km), dtype: float64

## RDS에 데이터 업로드

In [18]:
mdyr.to_sql(name='MainData_inner_예림', con=db_connection, if_exists='replace', index=False)